In [ ]:
import pandas as pd
import chardet
import re

In [ ]:
pattern_table = re.compile(r"<table[\s\S]+?>[\s\S]+?<\/table>")
pattern_td = re.compile(r"<td><span class=\"\w+\">(.+?)[\s]?</span></td>")

spoken_freq_nlt_with_enter_path = (
    "D:/05.japanproject/jp/freq/written_freq_nlt_with_enter.txt"
)
written_freq_nlt_with_enter_csv_path = (
    "D:/05.japanproject/jp/freq/written_freq_nlt_with_enter.csv"
)


lst = []
with open(written_freq_nlt_with_enter_path, "r", encoding="utf-8") as f:
    content = f.read()  # 一次性读取
    table_matches = pattern_table.findall(content)  # 或 finditer 迭代匹配
    for table_match in table_matches:
        td_matches_lst = pattern_td.findall(table_match)
        lst.append(td_matches_lst)
    columns = ["headword", "reading", "roman", "freq", "freq_per", "rank"]
    df = pd.DataFrame(lst, columns=columns)
print("ok")

In [ ]:
df.describe()

In [15]:
df.dropna(subset=["headword", "reading", "roman", "rank"], inplace=True)
# 2. 将次数映射到新列
value_counts = df["headword"].value_counts()
df["count"] = df["headword"].map(value_counts)

# mask = (df["type"] == "記号") | (df["headword"] == "*")
df.sort_values(by="rank", ascending=False, inplace=True)
df.to_csv(written_freq_nlt_with_enter_csv_path, index=False, encoding="utf-8-sig")
# df = df[~mask].copy()
df


,headword,reading,roman,freq,freq_per,rank,count
695,いる-非自立,イル,iru,"968,064",225.4396,0.99999,1
2545,こと,コト,koto,"701,470",163.3561,0.99998,1
5182,の,ノ,no,"686,025",159.7593,0.99996,1
3477,する,スル,suru,"601,911",140.1711,0.99995,72
5007,なる,ナル,naru,"484,861",112.9129,0.99994,1
...,...,...,...,...,...,...,...
85060,鼻する,ハナスル,hanasuru,1,0.0002,0.00005,1
85063,鼻っぱしら,ハナッパシラ,hanappashira,1,0.0002,0.00004,1
85086,鼻緒する,ハナオスル,hanaosuru,1,0.0002,0.00002,1
85089,鼻腔する,ビコウスル,bikousuru,1,0.0002,0.00001,1


In [ ]:
df["hrt"] = df["headword"] + "-" + df["reading"] + "-" + df["type"]
hrt_counts = df["hrt"].value_counts()
df["hrt_count"] = df["hrt"].map(hrt_counts)
df["r"] = (
    df.groupby("hrt")["rank"]
    .rank(
        method="first", ascending=False
    )  # 并列时按出现顺序排名（类似 SQL 的 ROW_NUMBER）  # 升序排列（默认）
    .astype(int)
)
df

In [ ]:
df[df["r"] == 1].to_csv(spoken_freq_nlb_csv_path, index=False, encoding="utf-8-sig")
# df.to_csv(spoken_freq_nlb_csv_path, index=False, encoding="utf-8-sig")
print("ok")

In [ ]:
def get_unicode_codepoints(text):
    """将文本中每个字符转换为4位十六进制Unicode码点字符串（无U+前缀）"""
    return "".join([f"{ord(char):04X}" for char in text])


# 创建示例 DataFrame
data = {
    "text_column": ["日本語", "こんにちは", "東京", "日本語"],  # 包含重复文本
    "value": [100, 200, 300, 400],
}
df = pd.DataFrame(data)

# 设置索引（处理重复值）
df.index = (
    df["text_column"].apply(get_unicode_codepoints)
    + "_"
    + df.groupby("text_column").cumcount().astype(str)
)

# 输出结果
print(df)

In [ ]:
with open("D:/05.japanproject/jp/freq/spoken_freq_nlb.csv", "rb") as f:
    result = chardet.detect(f.read())
    print(f"检测到编码: {result['encoding']}")